In [1]:
!pip install html5lib beautifulsoup4
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re
import pandas as pd
import numpy as np

In [2]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

In [16]:
# 인코딩이 euc-kr입니다. 저걸 지정하지 않으면 csv가 안 열려요!
# idol_list_withid를 불러오셔야 합니다. 멜론 url에 가수를 찾기 위한 id입니다. 

idol_list = pd.read_csv('idol_list_idfix.csv', encoding='euc-kr')
idol_list.head()

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id
0,서태지와 아이들,SeoTaiji And Boys,3,서태지,남,요요 기획,100161.0
1,더 블루,The Blue,2,손지창,남,SM 엔터테인먼트,180144.0
2,잼,Zam,5,조진수,혼,DSP 미디어,225786.0
3,에스오에스,S.O.S,4,NaN,여,삼포니 레코드,101645.0
4,솔리드,Solid,3,정재윤,남,비앤비,100037.0


In [17]:
# 멜론에 정보가 없어서 아예 페이지가 없었던 가수들이 몇몇 있습니다.
# 이런 경우 id가 공란이기에 수집이 불가합니다. 모두 드랍하겠습니다. 

idol_list = idol_list[np.isfinite(idol_list['idol_id'])].reset_index(drop=True)

In [18]:
# id가 flaot으로 저장되어 있습니다. int로 바꿔주고 저장하겠습니다. 
idol_list['idol_id'] = idol_list['idol_id'].astype(int)
idol_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 7 columns):
artist         332 non-null object
artist_eng     331 non-null object
member_num     332 non-null object
leader_name    274 non-null object
gender         331 non-null object
agency         313 non-null object
idol_id        332 non-null int32
dtypes: int32(1), object(6)
memory usage: 16.9+ KB


In [19]:
idol_list.idol_id.nunique()

332

In [20]:
idol_list[idol_list.idol_id.duplicated(keep=False)]

,artist,artist_eng,member_num,leader_name,gender,agency,idol_id


In [21]:
# 가수마다 한글로 된 가수 이름을 붙여주어야 하기에 가수 이름 리스트로 받아오기
names = idol_list.artist
names[0]

'서태지와 아이들'

In [22]:
# 크롤러가 링크를 찾을 때 매번 바꿔주는 값. for문에 사용됩니다. 
ids = idol_list.idol_id
ids[0]

100161

In [23]:
driver = webdriver.Chrome('C:/Users/pje17/Desktop/Lyrics/Crawlling/chromedriver')
driver.get("https://www.melon.com")

In [24]:
test = ['995169']

In [28]:
# 수집을 위한 빈 리스트 생성

artist = []
melon_artist = []
artist_m = []
debut_y = []
debut_t = []
fan = []

In [29]:
names[48:53]

48     에스지
49     파파야
50    제이티엘
51    비오케이
52    엑스라지
Name: artist, dtype: object

In [30]:
# 가수 정보 url을 가져와서 변수에 넣어주기

l1 = 'https://www.melon.com/artist/timeline.htm?artistId='
# url id

i = 0
  # 멤버명 반복문을 위해 만든 변수

for id in ids:
    
    driver.get(l1+str(id))    
    sleep(2)
    
    # 가수 이름 지정
    artist.append(names[i])
    
     # 멜론용 가수 이름 크롤링
    try:
        melon_artist.append(driver.find_element_by_class_name('title_atist').text)
    except NoSuchElementException as e:
        melon_artist.append('')
        
    # 멤버 리스트 크롤링
    mlist = []
    m = 2
    
    while True:
               
        try:
            mlist.append(driver.find_element_by_css_selector("div.wrap_atist_info > div > a:nth-child("+str(m)+ ")> span:nth-child(1)").text)
            m += 1
        except NoSuchElementException as e:
            break

    artist_m.append(mlist)
   
    
    # 데뷔 날짜 크롤링
    try:
        debut_y.append(driver.find_element_by_css_selector('div.wrap_atist_info > dl.atist_info.clfix > dd:nth-child(2) > span').text)
    except NoSuchElementException as e:
        debut_y.append('')

    # 데뷔곡 크롤링
    try:
        debut_t.append(driver.find_element_by_css_selector('div.wrap_atist_info > dl.atist_info.clfix > dd:nth-child(2) > a > span.songname12').text)
    except NoSuchElementException as e:
        debut_t.append('')
        
    # 팬맺기 수 크롤링
    fan.append(driver.find_element_by_xpath('//*[@id="d_like_count"]').text)
    
    print(melon_artist[i], debut_t[i], debut_y[i], artist_m[i], fan[i])
    i += 1
    
    
    

서태지와 아이들 난 알아요 1992.03 ['서태지', '양현석', '이주노'] 4,893
더 블루  1992 |너만을 느끼며 ['손지창', '김민종'] 474
잼   [] 19
S.O.S   [] 39
솔리드 이젠 나를 (Give Me A Chance) 1993.01 ['이준', '정재윤', '김조한'] 1,772
듀스 나를 돌아봐 1993.04 ['이현도', '김성재'] 2,125
노이즈 너에게 원한건 1992.11 [] 367
잉크   [] 9
룰라 100일(日)째 만남 1994.07 ['이상민', '고영욱', '김지현', '채리나'] 1,415
쿨 (COOL) 너이길 원했던 이유 1994.10 ['이재훈 (쿨)', '김성수', '유리'] 12,606
Jet Are You Gonna Be My Girl 2003.10 [] 852
터보 나 어릴 적 꿈 1995.07 ['김종국', '마이키', '김정남'] 9,394
R.ef 고요속의 외침 1995.03 ['이성욱', '성대현'] 612
스머프   [] 4
H.O.T. 전사의 후예 (폭력시대) 1996.09 ['토니 안', '문희준', '강타 (KANGTA)', '장우혁', '이재원'] 15,892
언타이틀  1996 |無題(무제) ['유건형', '서정환'] 478
Hamohamo  1996 |빠삐용 (그의 두통은 자유였다) ['이성진', '천명훈'] 94
아이돌   [] 144
구피 많이많이 1996.11 ['신동욱', '박성호 (구피)'] 381
태사자  1997.11 |도 [] 397
엔알지 (NRG) 할 수 있어 1997.10.28 ['천명훈', '노유민', '이성진'] 1,054
오피피에이  1998.01 |애국심(愛國心) [] 246
T.N.B  1997 |비키니-Ⅰ [] 35
젝스키스 학원별곡 (學園別曲) 1997.04.15 ['은지원', '이재진', '김재덕', '강성훈', '장수원'] 48,343
쎄쎄쎄  1996 |아미가르 레스토랑 [] 78
베이비 복스 Shee 2007.01 ['황연경']

브레이브걸스 아나요 2011.04 ['민영 (Brave Girls)', '유정', '은지', '유나', '하윤'] 2,801
Apink (에이핑크) 몰라요 2011.04 ['정은지', '손나은 (에이핑크)', '박초롱 (에이핑크)', '오하영 (에이핑크)', '김남주 (에이핑크)', '윤보미 (에이핑크)'] 138,929
씨스타19 Ma Boy (마보이) 2011.05 ['보라 (씨스타)', '효린'] 4,781
천상지희 다나&선데이 나 좀 봐줘 (One More Chance) 2011.07.11 ['다나 (DANA)', '선데이 (SUNDAY)'] 237
애프터스쿨 RED 밤 하늘에 2011.07 ['가희', '정아 (애프터스쿨)', '유이 (애프터스쿨)', '나나 (애프터스쿨)'] 378
애프터스쿨 BLUE 원더보이 (Wonder Boy) 2011.07 [] 573
블레이디 (Blady)  2011.05 |찌릿찌릿 ['예은', '다영', '가빈', '티나', '기루'] 2,227
스텔라 로켓걸 (Rocket Girl) (Feat. Eric) 2011.08.25 ['가영', '전율', '효은', '민희', '소영'] 5,570
씨리얼 No No No No No 2011.10 ['케미 (씨리얼)', '레디 (씨리얼)', '에피 (씨리얼)', '앤제이 (씨리얼)', '레니 (씨리얼)'] 220
B.A.P WARRIOR 2012.01 ['방용국', '젤로', '힘찬', '대현 (B.A.P)', '영재 (B.A.P)', '종업 (B.A.P)'] 27,989
NU`EST FACE 2012.03.15 ['JR (뉴이스트)', 'Aron (뉴이스트)', '백호 (뉴이스트)', '황민현', '렌 (뉴이스트)'] 89,984
비투비 비밀 (Insane) 2012.03 ['서은광', '이민혁', '이창섭', '임현식', '프니엘', '정일훈', '육성재'] 176,461
EXO 늑대와 미녀 (Wolf) 2013.06.03 ['카이 (KAI)', '첸 (CHEN)', '세훈',

써스포 (SUS4)   ['산하', '휘린', '아인 (써스포)'] 368
오마이걸 CUPID 2015.04.20 ['효정 (오마이걸)', '미미 (오마이걸)', '유아 (오마이걸)', '승희 (오마이걸)', '지호 (오마이걸)', '비니 (오마이걸)', '아린 (오마이걸)'] 44,172
멜로디데이 (MelodyDay) 어떤 안녕 2014.02.25 ['여은 (멜로디데이)', '예인 (멜로디데이)', '차희 (멜로디데이)', '유민 (멜로디데이)'] 7,352
에이프릴 (APRIL) 꿈사탕 2015.08.24 ['윤채경 (에이프릴)', '김채원 (에이프릴)', '이나은 (에이프릴)', '양예나 (에이프릴)', '레이첼 (에이프릴)', '이진솔 (에이프릴)'] 12,235
DIA (다이아)  2015.09.12 ['예빈 (다이아)', '유니스', '제니', '정채연', '기희현', '은채', '주은', '솜이 (다이아)'] 14,938
TWICE (트와이스) OOH-AHH하게 2015.10.20 ['나연', '정연', '모모', '사나', '지효', '미나', '다현', '채영', '쯔위'] 179,443
카드 (KARD) Oh NaNa (Hidden. 허영지) 2016.12.13 ['BM', 'J.seph', '전소민', '전지우'] 15,276
아스트로 (ASTRO)   ['차은우 (ASTRO)', '문빈 (ASTRO)', 'MJ (ASTRO)', '진진 (ASTRO)', '라키 (ASTRO)', '윤산하 (ASTRO)'] 24,338
임팩트 롤리팝 (Lollipop) 2016.01.27 ['웅재 (임팩트)', '이상 (임팩트)', '태호 (임팩트)', '지안 (임팩트)', '제업 (임팩트)'] 1,263
펜타곤 Gorilla 2016.10.10 ['후이 (펜타곤)', '진호 (펜타곤)', '홍석', '이던', '신원', '여원', '옌안', '유토', '키노', '우석 (펜타곤)'] 15,047
SF9 팡파레 (Fanfare) 2016.10.05 ['영빈

In [31]:
artist_info = pd.DataFrame([artist, melon_artist, artist_m, debut_y, debut_t, fan])
artist_info = artist_info.T
artist_info.head()

,0,1,2,3,4,5
0,서태지와 아이들,서태지와 아이들,"[서태지, 양현석, 이주노]",1992.03,난 알아요,"4,893"
1,더 블루,더 블루,"[손지창, 김민종]",1992 |너만을 느끼며,,474
2,잼,잼,[],,,19
3,에스오에스,S.O.S,[],,,39
4,솔리드,솔리드,"[이준, 정재윤, 김조한]",1993.01,이젠 나를 (Give Me A Chance),"1,772"


In [32]:
artist_info.columns = [['artist', 'melon_artist', 'artist_m', 'debut_y', 'debut_t', 'fan']]

In [33]:
artist_info

,artist,melon_artist,artist_m,debut_y,debut_t,fan
0,서태지와 아이들,서태지와 아이들,"[서태지, 양현석, 이주노]",1992.03,난 알아요,"4,893"
1,더 블루,더 블루,"[손지창, 김민종]",1992 |너만을 느끼며,,474
2,잼,잼,[],,,19
3,에스오에스,S.O.S,[],,,39
4,솔리드,솔리드,"[이준, 정재윤, 김조한]",1993.01,이젠 나를 (Give Me A Chance),"1,772"
5,듀스,듀스,"[이현도, 김성재]",1993.04,나를 돌아봐,"2,125"
6,노이즈,노이즈,[],1992.11,너에게 원한건,367
7,잉크,잉크,[],,,9
8,룰라,룰라,"[이상민, 고영욱, 김지현, 채리나]",1994.07,100일(日)째 만남,"1,415"
9,쿨,쿨 (COOL),"[이재훈 (쿨), 김성수, 유리]",1994.10,너이길 원했던 이유,"12,606"


In [34]:
artist_info.debut_t.nunique()

debut_t    217
dtype: int64

In [35]:
artist_info[artist_info.melon_artist.duplicated(keep=False)]

,artist,melon_artist,artist_m,debut_y,debut_t,fan


In [36]:
artist_info.to_csv('artist_info.csv')